In [1]:
with open('prod.ini', 'w') as prod, open('dev.ini', 'w') as dev:
    prod.write('[Database]\n')
    prod.write('db_host=prod.mynetwork.com\n')
    prod.write('db_name=my_database\n')
    prod.write('\n[Server]\n')
    prod.write('port=8080\n')
    
    dev.write('[Database]\n')
    dev.write('db_host=dev.mynetwork.com\n')
    dev.write('db_name=my_database\n')
    dev.write('\n[Server]\n')
    dev.write('port=3000\n')

In [2]:
import configparser

In [3]:
class Config:
    def __init__(self, env = 'dev'):
        print(f'loading config from {env} file')
        config = configparser.ConfigParser()
        file_name = f'{env}.ini'
        config.read(file_name)
        self.db_host = config['Database']['db_host']
        self.db_name = config['Database']['db_name']
        self.port = config['Server']['port']
        

In [4]:
config = Config('dev')

loading config from dev file


In [5]:
config.__dict__

{'db_host': 'dev.mynetwork.com', 'db_name': 'my_database', 'port': '3000'}

In [6]:
class Config:
    def __init__(self, env = 'dev'):
        print(f'loading config from {env} file')
        config = configparser.ConfigParser()
        file_name = f'{env}.ini'
        config.read(file_name)
        for section_name in config.sections():
            for k, v in config[section_name].items():
                setattr(self, k, v)

In [7]:
config = Config('prod')

loading config from prod file


In [8]:
vars(config)

{'db_host': 'prod.mynetwork.com', 'db_name': 'my_database', 'port': '8080'}

In [10]:
class Section:
    def __init__(self, name, item_dict):
        """
        name: str
            name of the section
        item_dict: dict
            dictionary of named config values
        """
        self.name = name
        for k, v in item_dict.items():
            setattr(self, k, v)
        

In [11]:
class Config:
    def __init__(self, env = 'dev'):
        print(f'loading config from {env} file')
        config = configparser.ConfigParser()
        file_name = f'{env}.ini'
        config.read(file_name)
        for section_name in config.sections():
            section = Section(section_name, config[section_name])
            setattr(self, section_name.casefold(), section)

In [12]:
config = Config()

loading config from dev file


In [13]:
vars(config)

{'database': <__main__.Section at 0x261f323a940>,
 'server': <__main__.Section at 0x261f3ec2340>}

In [14]:
class SectionType(type):
    def __new__(cls, name, bases, cls_dict, section_name, item_dict):
        cls_dict['__doc__'] = f'config for {section_name} section'
        cls_dict['section_name'] = section_name
        for k , v in item_dict.items():
            cls_dict[k] = v
        return super().__new__(cls,name, bases, cls_dict)

In [16]:
class DatabaseSection(metaclass = SectionType, 
                      section_name = 'database', 
                      item_dict = {'db_host': 'db_host', 'db_name': 'db_name'}):
    pass

In [17]:
vars(DatabaseSection)

mappingproxy({'__module__': '__main__',
              '__doc__': 'config for database section',
              'section_name': 'database',
              'db_host': 'db_host',
              'db_name': 'db_name',
              '__dict__': <attribute '__dict__' of 'DatabaseSection' objects>,
              '__weakref__': <attribute '__weakref__' of 'DatabaseSection' objects>})

In [18]:
help(DatabaseSection)

Help on class DatabaseSection in module __main__:

class DatabaseSection(builtins.object)
 |  config for database section
 |  
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)
 |  
 |  ----------------------------------------------------------------------
 |  Data and other attributes defined here:
 |  
 |  db_host = 'db_host'
 |  
 |  db_name = 'db_name'
 |  
 |  section_name = 'database'



In [19]:
MyClass = type('MyClass', (object,), {})

In [20]:
MyClass

__main__.MyClass

In [21]:
type(MyClass)

type

In [24]:
MySection = SectionType('DBSection', (object,), {}, section_name = 'database',
                        item_dict = {'db_host': 'db_host', 'db_name': 'db_name'})

In [25]:
MySection

__main__.DBSection

In [26]:
vars(MySection)

mappingproxy({'__doc__': 'config for database section',
              'section_name': 'database',
              'db_host': 'db_host',
              'db_name': 'db_name',
              '__module__': '__main__',
              '__dict__': <attribute '__dict__' of 'DBSection' objects>,
              '__weakref__': <attribute '__weakref__' of 'DBSection' objects>})

In [27]:
help(MySection)

Help on class DBSection in module __main__:

class DBSection(builtins.object)
 |  config for database section
 |  
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)
 |  
 |  ----------------------------------------------------------------------
 |  Data and other attributes defined here:
 |  
 |  db_host = 'db_host'
 |  
 |  db_name = 'db_name'
 |  
 |  section_name = 'database'



In [28]:
class ConfigType(type):
    def __new__(cls, name, bases, cls_dict, env):
        """
        env: str
            the en we are loading for config like prod or dev
        """
        
        cls_dict['__doc__'] = f'Config for {env}'
        cls_dict['env'] = env
        config = configparser.ConfigParser()
        file_name = f'{env}.ini'
        config.read(file_name)
        for section_name in config.sections():
            class_name = section_name.capitalize()
            class_attr_name = section_name.casefold()
            section_items = config[section_name]
            bases = (object,)
            section_cls_dict = {}
            Section = SectionType(
            class_name, bases, section_cls_dict, section_name=section_name,item_dict=section_items)
            cls_dict[class_attr_name] = Section
        return super().__new__(cls, name, bases, cls_dict)
            

In [29]:
class DevConfig(metaclass = ConfigType, env = 'dev'):
    pass

class ProdConfig(metaclass = ConfigType, env = 'prod'):
    pass

In [30]:
DevConfig.database.db_name

'my_database'

In [31]:
ProdConfig.server.port

'8080'

In [32]:
help(DevConfig)

Help on class DevConfig in module __main__:

class DevConfig(builtins.object)
 |  Config for dev
 |  
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)
 |  
 |  ----------------------------------------------------------------------
 |  Data and other attributes defined here:
 |  
 |  database = <class '__main__.Database'>
 |      config for Database section
 |  
 |  
 |  env = 'dev'
 |  
 |  server = <class '__main__.Server'>
 |      config for Server section



In [33]:
help(DevConfig.database)

Help on class Database in module __main__:

class Database(builtins.object)
 |  config for Database section
 |  
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)
 |  
 |  ----------------------------------------------------------------------
 |  Data and other attributes defined here:
 |  
 |  db_host = 'dev.mynetwork.com'
 |  
 |  db_name = 'my_database'
 |  
 |  section_name = 'Database'

